In [3]:
pwd

'C:\\Users\\dandi\\Documents\\study2023\\small_app'

In [2]:
import os
os.chdir('C:\\Users\\dandi\\Documents\\study2023\\small_app')

In [4]:
from src.get_data import read_params
import argparse
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib
import os

def log_production_model(config_path):
    config = read_params(config_path)
    mlflow_config = config["mlflow_config"]

    model_name = mlflow_config["registered_model_name"]
    
    remote_server_uri = mlflow_config["remote_server_uri"]
    mlflow.set_tracking_uri(remote_server_uri)

    runs = mlflow.search_runs([1])
    lowest = runs["metrics.mae"].sort_values(ascending=True)[0]
    lowest_run_id = runs[runs["metrics.mae"] == lowest]["run_id"][0]
    
    client = MlflowClient()
    for mv in client.search_model_versions(f"name = '{model_name}'"):
        mv = dict(mv)
        pprint(mv, indent=4)

        
log_production_model('params.yaml')

{   'creation_timestamp': 1676893011228,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1676893011228,
    'name': 'ElasticNetWineModel',
    'run_id': '5ae3c66aae784d74add6a2cd920139bd',
    'run_link': '',
    'source': './artifacts/1/5ae3c66aae784d74add6a2cd920139bd/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '6'}
{   'creation_timestamp': 1676892751406,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1676892751406,
    'name': 'ElasticNetWineModel',
    'run_id': 'd7d3d9652782452f8d62ac55a545657e',
    'run_link': '',
    'source': './artifacts/1/d7d3d9652782452f8d62ac55a545657e/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '5'}
{   'creation_timestamp': 1676892441522,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1676892441522,
    'nam

In [16]:
from src.get_data import read_params
import argparse
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib
import os

def log_production_model(config_path):
    config = read_params(config_path)
    mlflow_config = config["mlflow_config"]

    model_name = mlflow_config["registered_model_name"]
    
    remote_server_uri = mlflow_config["remote_server_uri"]
    #mlflow.set_registry_uri(remote_server_uri)
    mlflow.set_tracking_uri(remote_server_uri)

    #runs = mlflow.search_runs(experiment_ids=1)
    runs = mlflow.search_runs([1])
    print(mlflow.search_runs([1]))
    
    
    lowest = runs["metrics.mae"].sort_values(ascending=True)[0]
    lowest_run_id = runs[runs["metrics.mae"] == lowest]["run_id"][0]
    
    client = MlflowClient()
    for mv in client.search_model_versions(f"name = '{model_name}'"):
        mv = dict(mv)

        if mv["run_id"] == lowest_run_id:
            current_version = mv["version"]
            logged_model = mv["source"]
            pprint(mv, indent=4)
            client.transition_model_version_stage(
                name = model_name,
                version = current_version,
                stage = "Production"
            )
        else:
            current_version = mv["version"]
            logged_model = mv["source"]
            client.transition_model_version_stage(
                name = model_name,
                version = current_version,
                stage = "Staging" #if production model already exists
                                
            )
            pprint(mv, indent=4)
    
    #loaded_model = mlflow.pyfunc.load_model(logged_model)
    #model_path = config["webapp_model_dir"]

    #joblib.dump(loaded_model, model_path) 
log_production_model("params.yaml")

if __name__=="__main__":
    args = argparse.ArgumentParser()
    args.add_argument("--config", default="params.yaml")
    parsed_args = args.parse_args()
    data = log_production_model(config_path=parsed_args.config)


                             run_id experiment_id    status  \
0  5ae3c66aae784d74add6a2cd920139bd             1  FINISHED   
1  d7d3d9652782452f8d62ac55a545657e             1  FINISHED   
2  463aa179a5dc405db7aa80e782f13646             1  FINISHED   
3  02c3c3bf4a8d4e34a42a50dfc0ded5c0             1  FINISHED   
4  32942168bb8348b18f675bf7c3c3b85e             1  FINISHED   
5  7003a859f7754839878e91eb764e5b7d             1  FINISHED   

                                        artifact_uri  \
0  ./artifacts/1/5ae3c66aae784d74add6a2cd920139bd...   
1  ./artifacts/1/d7d3d9652782452f8d62ac55a545657e...   
2  ./artifacts/1/463aa179a5dc405db7aa80e782f13646...   
3  ./artifacts/1/02c3c3bf4a8d4e34a42a50dfc0ded5c0...   
4  ./artifacts/1/32942168bb8348b18f675bf7c3c3b85e...   
5  ./artifacts/1/7003a859f7754839878e91eb764e5b7d...   

                        start_time                         end_time  \
0 2023-02-20 11:36:48.443000+00:00 2023-02-20 11:36:51.238000+00:00   
1 2023-02-20 11:32:28.6

usage: ipykernel_launcher.py [-h] [--config CONFIG]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\dandi\AppData\Roaming\jupyter\runtime\kernel-30b8f367-1ce9-4ea2-8298-0ccad3877551.json


SystemExit: 2

C:\Users\dandi\anaconda3\envs\wineq\lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
